# Teradata Example
## 1. Import Modules

In [ ]:
# Basic modules
import pandas as pd
import numpy
from getpass import getuser, getpass
import datetime as dt

# Teradata modules
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR
import teradatasqlalchemy
import teradataml as tdml
import teradatasql
from teradataml.context.context import *
from teradataml.dataframe.copy_to import copy_to_sql

# Display Options
from IPython.display import HTML
pd.set_option('display.max_columns', None)

## 2. Setup Connection

In [ ]:
tdhost = ''
username = getuser()
passwrd = getpass('password(%s): ' % username)
con_str = f'teradata://{username}:{passwrd}@{tdhost}/?logmech=LDAP'

## 3. `teradataml`

In [ ]:
# Create connection
con = create_engine(con_str)
create_context(tdsqlengine = con)

In [ ]:
# Download table
qry = """
select top 100 from table;
"""
df_td = tdml.DataFrame.from_query(qry)

# Convert to DF
df_pd = df_td.to_pandas()

# Upload DF
copy_to_sql(
    df = df_pd,
    schema = '',
    table_name = '',
    primary_index = '',
    if_exists='replace'
)

# Upadte tables
get_context().execute('delete from table all')

# Aggregation
df_td.select(['column1', 'column2', 'column3'])\
    .groupby(['column1', 'column2'])\
    .agg(['column3' : ['count', 'min', 'max']])\
    .sort(['column1', 'column2'])

## 4. SQL Magic
Load extensions

In [ ]:
%load_ext sql
%sql $con_str

In [ ]:
%%sql df <<
select top 100 from table;

In [ ]:
# Inline queries
df_inline = %sql select top 100 from table
df_inline.DataFrame().head()

# Passing variables
test = '123'
%sql select :test as col

## 5. `teradatasql`

In [ ]:
# Create connection
con = teradatasql.connect(
    None,
    host = tdhost,
    user = username,
    password = passwrd,
    logmech = 'LDAP'
    )

# Download data to DF
df = pd.read_sql('select top 100 from table', con)

In [ ]:
# Teradata curson
with teradatasql.connect(
    None,
    host = tdhost,
    user = username,
    password = passwrd,
    logmech = 'LDAP'
    ) as con:

    with con.cursor() as cur:
        qry = """
        select top 100 from table;
        """
        cur.execute(qry)
        data_ = cur.fetchall() # fetch all the data from the last script if there are multiple script included
        print(cur.rowcount) # print the last executed script rowcount

In [ ]:
# Table operations
df.to_sql(
    'tableName',
    con = con,
    if_exists = 'replace',
    index = False,
    dtype = {'column1': VARCHAR(100)} # Need to set type for first column for index creation
    )

In [ ]:
# Closing connection
con.close()